<a href="https://colab.research.google.com/github/rafaelferraz83/DeepRecommender/blob/master/Trabalho_puc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DESENVOLVIMENTO DE REGRAS DE ASSOCIAÇÃO PARA UM VAREJO UTILIZANDO O MODELO APRIORI



Nicolau Assumpção Branco

Departamento de Informática

Pontifícia Universidade Católica do Rio de Janeiro (PUC-Rio)

Rio de Janeiro RJ, Brasil

nabranco@gmail.com

Tatiana Escovedo

Departamento de Informática

Pontifícia Universidade Católica do Rio de Janeiro (PUC-Rio)

Rio de Janeiro RJ, Brasil

tatiana@inf.puc-rio.br

Inicialmente se instalou a biblioteca que permitirá importar os algoritmos *Apriori* e *Rules Association*.

In [ ]:
pip install mlxtend

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Seguindo, foi usada a declaração *import* onde forma importadas as bibliotecas *pandas* e *numpy*, além disso importamtos da biblioteca os algoritmos que serão usadados para estruturar as regras de associação.

In [ ]:
import numpy as np 
import pandas as pd 
from mlxtend.frequent_patterns import apriori, association_rules

Após essa etapa, foram trazidas as informações que serão analisadas através do comando da biblioteca pandas - *pd_read_excel*. Os arquivos foram retirados de uma base do SQL Server e tiveram um primeiro tratamento dentro da própria ferramente. Ademais esse comando possibilita que a informação seja trazida como um *dataframe*.

In [ ]:
#Comando roadado no SQL Server que permitiu selecionar as instâncias 
'''
DECLARE @inicio DATE; 
DECLARE @fim DATE; 
SET @inicio = '2021-11-01'; 
SET @fim = '2022-10-31';

SELECT TOP 1000000 T.DT_DATA,
                   T.CLIENTE,
                   T.LINHA,
                   T.PRODUTO,
                   T.CANAL,
                   SUM(T.PEDIDOS) AS PEDIDOS,
                   SUM(T.VALOR) AS [PREÇO DO PRODUTO]
                   FROM (SELECT CAL_DT_DATA AS DT_DATA,
                                A.CLI_SK_CLIENTE AS CLIENTE,
                                E.CLI_DS_PRIMEIRO_NOME AS NOME,
                                E.CLI_DS_UF AS UF,
                                E.CLI_DS_SEXO AS SEXO,
                               (CASE 
                                  WHEN LEN(E.CLI_NU_DATA_NASCIMENTO) = 8 THEN SUBSTRING(CAST(E.CLI_NU_DATA_NASCIMENTO AS VARCHAR), 1, 4)
                                                                        +'-'+ SUBSTRING(CAST(E.CLI_NU_DATA_NASCIMENTO AS VARCHAR), 5, 2)
                                                                        +'-'+ SUBSTRING(CAST(E.CLI_NU_DATA_NASCIMENTO AS VARCHAR), 7,2) 
                                  ELSE NULL
                                END) AS DT_DATA_NASCIMENTO,
                                S.CLASSE_SOCIAL_ENR AS CLASSE_SOCIAL,
                                C.LIN_DS_LINHA AS LINHA, 
                                F.PRD_DS_PRODUTO AS PRODUTO, 
                                D.TAB_DS_CANAL_II_5 AS CANAL,
                                COUNT(DISTINCT CAL_DT_DATA) AS PEDIDOS, 
                                SUM(VND_VL_NOTAFISCAL) AS VALOR 
                         FROM fat.bfs_vendas AS A INNER JOIN DIM.BFS_CALENDARIO AS B ON A.VND_DT_VENDA = B.CAL_DA_ID 
                                                  LEFT JOIN DIM.BFS_LINHA AS C ON A.LIN_SK_LINHA = C.LIN_SK_LINHA 
                                                  INNER JOIN DIM.BFS_LOJA AS D ON A.TAB_SK_CODIGO = D.TAB_SK_TABELA 
                                                  LEFT JOIN DIM.BFS_CLIENTE AS E ON A.CLI_SK_CLIENTE = E.CLI_SK_CLIENTE 
                                                  LEFT JOIN DIM.BFS_PRODUTO AS F ON A.PRD_SK_PRODUTO = F.PRD_SK_PRODUTO
                                                  LEFT JOIN CRM.CRM_SERASA_ENR S ON E.CLI_DS_CGC_AJTD = S.cpf
                         WHERE 1=1 
                               AND A.CLI_SK_CLIENTE NOT IN (6999157)
                               AND UPPER(LIN_DS_LINHA) <> 'ASSINATURA FASTPRIME' 
                               AND (CLI_DS_PESSOA NOT IN ('J') OR CLI_DS_PESSOA IS NULL) 
                               AND B.CAL_DT_DATA BETWEEN @inicio AND @fim 
                               AND A.CLI_SK_CLIENTE > 0
                               AND D.TAB_DS_CANAL_II_5 NOT LIKE '%Fast Club%'
                               AND D.TAB_DS_CANAL_II_5 NOT LIKE '%Lista de Casamento%'
                               AND D.TAB_DS_CANAL_II_5 NOT LIKE '%AJ-Matriz%'
                               AND D.TAB_DS_CANAL_II_5 NOT LIKE '%Outlet%'
                               AND D.TAB_DS_CANAL_II_5 NOT LIKE '%Corporativo%'
                               AND D.TAB_DS_CANAL_II_5 NOT LIKE '%N/A%'
                               AND D.TAB_DS_CANAL_II_5 NOT LIKE '%Premium%'
                               AND C.LIN_DS_LINHA NOT LIKE '%INSTALACAO%'
                               AND C.LIN_DS_LINHA NOT LIKE '%GARANTIA%'
                               AND C.LIN_DS_LINHA NOT LIKE '%SEGURO%'
                               AND C.LIN_DS_LINHA NOT LIKE '%SUPORTE DIGITAL 24H%'
                               AND C.LIN_DS_LINHA NOT LIKE 'SEGURO E QUEBRA'
                               AND C.LIN_DS_LINHA NOT LIKE 'QUEBRA ACIDENTAL'
                               AND C.LIN_DS_LINHA NOT LIKE 'MOVEIS E ACESSORIOS'
                               AND C.LIN_DS_LINHA NOT LIKE 'IMAGENS DIGITAIS'
                               AND C.LIN_DS_LINHA NOT LIKE 'NOVAS LINHAS'
                               AND C.LIN_DS_LINHA NOT LIKE 'CARTAO POSA'
                               AND C.LIN_DS_LINHA NOT LIKE 'ILUMINACAO'
                               AND C.LIN_DS_LINHA NOT LIKE 'FATURAMENTO 3P'
                               AND C.LIN_DS_LINHA NOT LIKE 'N/A'
                               AND C.LIN_DS_LINHA NOT LIKE 'ESPORTES'
                               AND C.LIN_DS_LINHA NOT LIKE 'AUTOMACAO RESIDENCIA'
                               AND C.LIN_DS_LINHA NOT LIKE 'KIDS'
                               AND C.LIN_DS_LINHA NOT LIKE 'BEBIDAS DIVERSAS'
                               AND C.LIN_DS_LINHA NOT LIKE 'INOVACAO'
                               AND C.LIN_DS_LINHA NOT LIKE 'BRINQUEDOS'
                               AND C.LIN_DS_LINHA NOT LIKE 'TRATAMENTO DE AR' 
                               AND F.PRD_DS_PRODUTO NOT LIKE '%CONTROLE REMOTO%'
                               AND F.PRD_DS_PRODUTO NOT LIKE '%PELICULA%'
                               AND F.PRD_DS_PRODUTO NOT LIKE '%CHIP%'
                         GROUP BY CAL_DT_DATA,
                                  A.CLI_SK_CLIENTE,
                                  E.CLI_DS_PRIMEIRO_NOME,
                                  E.CLI_DS_UF,
                                  E.CLI_DS_SEXO,
                                 (CASE 
                                    WHEN LEN(CLI_NU_DATA_NASCIMENTO) = 8 THEN SUBSTRING(CAST(CLI_NU_DATA_NASCIMENTO AS VARCHAR), 1, 4) 
                                                                        +'-'+ SUBSTRING(CAST(CLI_NU_DATA_NASCIMENTO AS VARCHAR), 5, 2) 
                                                                        +'-'+ SUBSTRING(CAST(CLI_NU_DATA_NASCIMENTO AS VARCHAR), 7,2) 
                                    ELSE NULL
                                  END),
                                  S.CLASSE_SOCIAL_ENR,
                                  C.LIN_DS_LINHA, 
                                  F.PRD_DS_PRODUTO, 
                                  D.TAB_DS_CANAL_II_5) AS T 
WHERE 1=1
GROUP BY T.DT_DATA,
         T.CLIENTE,
         T.LINHA,
         T.PRODUTO,
         T.CANAL
HAVING SUM(T.VALOR) >= 100
ORDER BY NEWID()
'''

In [ ]:
df = pd.read_excel('dados_puc.xlsx')

Após a realziação do upload do arquivo em excel com as 1 milhão de instâncias, foram realizados alguns comandos básicos do *pandas* para verificação da integridade da informação.

1. Visualização das primeiras dez linhas;
2. Verificação das colunas indexadas no *dataframe*;
3. Verificação distinsta dos tipos de canais pertecentes ao atributo Canal no *dataframe*; trazidos pelas informações;
4. Cópia do *dataframe* para o df_apriori onde será realizada a modelagem dos dados através da aplicação dos dois algorimtos destacados na introdução;
5. Aplicação na cópia do dataframe das funções numpy combinadas isnan() e sum(), para a verificação através da contagem das colunas do *dataframe* para valores ausentes.   

In [ ]:
df.head(10)

,DT_DATA,CLIENTE,LINHA,PRODUTO,CANAL,PEDIDOS,VALOR
0,2021-12-28,11081650,UTILIDADES DOMESTICA,ASPIR.PO 60W 2 EM 1 VSA15B CNB,Site,1,598.99
1,2021-12-30,3744024,APPLE,IPHONE SE 128GB BCO B,Site,1,2999.00
2,2022-03-15,4869923,VIDEO,TV 4K CRYSTAL UHD 55P. SMARTB,Lojas,1,2789.00
3,2022-09-16,13920645,APPLE,IPHONE 13 128GB PTO B,Site,1,4899.00
4,2022-06-12,2382537,ACESSORIOS,CARREGADOR P/IPH 2 SAIDAS BCO,Lojas,1,169.00
5,2022-04-29,2676237,APPLE,IPHONE 13 PRO MAX 256GB VRDB,Site,1,7699.00
6,2022-09-07,5602619,GAMES,VOLANTE DRIVING FORCE G29 PTO,Lojas,1,1613.60
7,2022-04-14,2306471,INFORMATICA,NB I7 16GB/512GB SSD/IRISXE 1B,Lojas,1,7643.00
8,2022-05-31,1564,LINHA BRANCA,COOKTOP GAS 4B. VIDRO PTOB,Lojas,1,729.00
9,2021-12-29,5876295,APPLE,IPHONE 13 PRO MAX 256GB DRDB,Site,1,9499.00


In [ ]:
df.columns

Index(['DT_DATA', 'CLIENTE', 'LINHA', 'PRODUTO', 'CANAL', 'PEDIDOS', 'VALOR'], dtype='object')

In [ ]:
df.CANAL.unique() 

array(['Site', 'Lojas', 'Televendas'], dtype=object)

In [ ]:
df_apriori = df

In [ ]:
df_apriori.head()

,DT_DATA,CLIENTE,LINHA,PRODUTO,CANAL,PEDIDOS,VALOR
0,2021-12-28,11081650,UTILIDADES DOMESTICA,ASPIR.PO 60W 2 EM 1 VSA15B CNB,Site,1,598.99
1,2021-12-30,3744024,APPLE,IPHONE SE 128GB BCO B,Site,1,2999.00
2,2022-03-15,4869923,VIDEO,TV 4K CRYSTAL UHD 55P. SMARTB,Lojas,1,2789.00
3,2022-09-16,13920645,APPLE,IPHONE 13 128GB PTO B,Site,1,4899.00
4,2022-06-12,2382537,ACESSORIOS,CARREGADOR P/IPH 2 SAIDAS BCO,Lojas,1,169.00


In [ ]:
df.isna().sum()

DT_DATA    0
CLIENTE    0
LINHA      0
PRODUTO    0
CANAL      0
PEDIDOS    0
VALOR      0
dtype: int64

###LIMPEZA DOS DADOS

Passada a primeira etapa vamos nos devbruçar sobre a preparação do *dataframe* para que se consiga rodar o algoritmo *Apriori*.
Logo de início usamos o método do pandas *str.strip()* para que sejam removidos os espaços extras da coluna do atributo Linhas.
Fizemos uma pequena verificação da quantidade instãncias, apenas para ratificação.
Após, chegamos em um parte bem importante que antecede a criação do modelo, pois definimos uma divisão para baseada nos canais de vendas e cada canal possuía uma certa quantidade de transação.
Finalmente, verificamos a incidência de cada Linha em relação as transações através também da contagem.

In [ ]:
#Removendo espaços extras na descrição
#df_apriori['PRODUTO'] = df_apriori['PRODUTO'].str.strip()
df_apriori['LINHA'] = df_apriori['LINHA'].str.strip()

In [ ]:
df_apriori.head(10)

,DT_DATA,CLIENTE,LINHA,PRODUTO,CANAL,PEDIDOS,VALOR
0,2021-12-28,11081650,UTILIDADES DOMESTICA,ASPIR.PO 60W 2 EM 1 VSA15B CNB,Site,1,598.99
1,2021-12-30,3744024,APPLE,IPHONE SE 128GB BCO B,Site,1,2999.00
2,2022-03-15,4869923,VIDEO,TV 4K CRYSTAL UHD 55P. SMARTB,Lojas,1,2789.00
3,2022-09-16,13920645,APPLE,IPHONE 13 128GB PTO B,Site,1,4899.00
4,2022-06-12,2382537,ACESSORIOS,CARREGADOR P/IPH 2 SAIDAS BCO,Lojas,1,169.00
5,2022-04-29,2676237,APPLE,IPHONE 13 PRO MAX 256GB VRDB,Site,1,7699.00
6,2022-09-07,5602619,GAMES,VOLANTE DRIVING FORCE G29 PTO,Lojas,1,1613.60
7,2022-04-14,2306471,INFORMATICA,NB I7 16GB/512GB SSD/IRISXE 1B,Lojas,1,7643.00
8,2022-05-31,1564,LINHA BRANCA,COOKTOP GAS 4B. VIDRO PTOB,Lojas,1,729.00
9,2021-12-29,5876295,APPLE,IPHONE 13 PRO MAX 256GB DRDB,Site,1,9499.00


In [ ]:
df_apriori.shape

(1000000, 7)

In [ ]:
df_apriori['CANAL'].value_counts()

Lojas         631569
Site          354750
Televendas     13681
Name: CANAL, dtype: int64

In [ ]:
df_apriori['LINHA'].value_counts()

APPLE                   215246
ACESSORIOS              146060
AUDIO                   134570
LINHA BRANCA            118022
UTILIDADES DOMESTICA    110210
VIDEO                    99452
CELULAR                  81947
INFORMATICA              58700
GAMES                    29263
UTENSILIOS DOMESTICO      6530
Name: LINHA, dtype: int64

DIVISÃO DOS DADOS DE ACORDO COM OS CANAIS DE VENDA

In [ ]:
#Transações realizadas nas lojas
#basket_lojas = (df_apriori[df_apriori['CANAL'] =="Lojas"] 
#		.groupby(['CLIENTE', 'PRODUTO'])['PEDIDOS'] 
#		.sum().unstack().reset_index().fillna(0) 
#		.set_index('CLIENTE')) 


Neste momento, iremos realizar a separação das transações agrupando o CLIENTE e as Linhas através da soma do PEDIDOS, seguindo aplicação doo método *unstrack()* que retorná um novo *dataframe* com os rótulos CLIENTE e LINHA, foi redefinido o índice do *dataframe* (*reset_index()*), reforçamos a susbtituíção dos objetos *NaN* por zero com o método *fillna(0)* e definimos usando o *set_index* os CLIENTES como índice.
A saída será dada por um *dataframe* específico para cada canal.

In [ ]:
#Transações realizadas nas lojas
basket_lojas = (df_apriori[df_apriori['CANAL'] =="Lojas"] 
		.groupby(['CLIENTE', 'LINHA'])['PEDIDOS'] 
		.sum().unstack().reset_index().fillna(0) 
		.set_index('CLIENTE')) 

In [ ]:
basket_lojas.head()

LINHA,ACESSORIOS,APPLE,AUDIO,CELULAR,GAMES,INFORMATICA,LINHA BRANCA,UTENSILIOS DOMESTICO,UTILIDADES DOMESTICA,VIDEO
CLIENTE,,,,,,,,,,
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
65,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
68,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
76,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
#Transações realizadas pelo site
#		.groupby(['CLIENTE', 'PRODUTO'])['PEDIDOS'] 
#		.sum().unstack().reset_index().fillna(0) 
#		.set_index('CLIENTE'))

In [ ]:
#Transações realizadas nas site
basket_site = (df_apriori[df_apriori['CANAL'] =="Site"] 
		.groupby(['CLIENTE', 'LINHA'])['PEDIDOS'] 
		.sum().unstack().reset_index().fillna(0) 
		.set_index('CLIENTE')) 

In [ ]:
basket_site.head()

LINHA,ACESSORIOS,APPLE,AUDIO,CELULAR,GAMES,INFORMATICA,LINHA BRANCA,UTENSILIOS DOMESTICO,UTILIDADES DOMESTICA,VIDEO
CLIENTE,,,,,,,,,,
11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
69,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
115,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
129,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
149,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#Transações realizadas nas televendas
basket_televendas = (df_apriori[df_apriori['CANAL'] =="Televendas"] 
		.groupby(['CLIENTE', 'LINHA'])['PEDIDOS'] 
		.sum().unstack().reset_index().fillna(0) 
		.set_index('CLIENTE')) 

In [ ]:
basket_televendas.head()

LINHA,ACESSORIOS,APPLE,AUDIO,CELULAR,GAMES,INFORMATICA,LINHA BRANCA,UTENSILIOS DOMESTICO,UTILIDADES DOMESTICA,VIDEO
CLIENTE,,,,,,,,,,
170,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
230,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
644,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
715,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
731,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


TRANSFORMAÇÃO DOS DADOS USANDO HOT ENCODING

Pegamos os três dataframes de saída da divisão por canal e aplicamos a função definida como *hot_encoding* (codificação *one hot encoding*), com o intuito de codificar e denotar os valores assumidos pelas variáveis categóricas.

In [ ]:
# Definindo função simples de hot encoding para sinalizar se o item foi ou não comprado
def hot_encode(x): 
	if(x<= 0): 
		return 0
	if(x>= 1): 
		return 1

# Codificando os datasets 
basket_encoded = basket_lojas.applymap(hot_encode) 
basket_lojas = basket_encoded 

basket_encoded = basket_site.applymap(hot_encode) 
basket_site = basket_encoded

basket_encoded = basket_televendas.applymap(hot_encode) 
basket_televendas = basket_encoded

CONSTRUINDO OS MODELOS E REALIZANDO A ANÁLISE DOS RESULTADOS

In [ ]:
# Suporte: a frequência de um item sendo comprado em toda a transação (quantas vezes um produto está sendo comprado). 
# Este dataset tem 7500 transações e escolhi considerar itens que estavam em pelo menos 30 transações. Portanto, o suporte deve ser 30/7500 = 0,004.
# Suporte (Produto A) = Nº de transações com A / No. total de transações

# Confiança: a frequência de compra de um produto junto com outro produto (quantas vezes compramos produtos juntos, dividido pelo número de vezes que um produto é comprado).
# Exemplo: compramos o "Produto A" 100 vezes e, dentre essas 100 transações, compramos o "Produto B" 60 vezes. 
# Então, a confiança (comprar o "Produto B" se um cliente comprar o "Produto A") é 0,6. Aqui, definimos 0,4 como o mínimo confiante.
# Confiança (Produto A -> Produto B) = Nº de transações (A-> B) / Nº de transações (A)

# Lift: o "aumento" que recebemos ao sugerir um produto para uma pessoa que comprou outro produto e para uma pessoa normal que não comprou o outro produto. 
# Exemplo: digamos que as pessoas tenham comprado o "Produto B" 30 vezes e o "Produto A" 500 vezes, sendo 25 das 30 transações do "Produto B" também incluem o "Produto A". 
# Assim, a porcentagem de pessoas que compram o "Produto B" (Suporte B) é 30/7500 = 0,004 e a 
# porcentagem de pessoas que compram o "Produto B" e o "Produto A" é 25/500 = 0,05. 
# Portanto, o aumento (lift) obtido é de 0,05/0,004 = 12,5%.
# Lift (Produto A -> Produto B) = Confiança (A -> B) / Suporte (B)

In [ ]:
basket_lojas.head(5)

LINHA,ACESSORIOS,APPLE,AUDIO,CELULAR,GAMES,INFORMATICA,LINHA BRANCA,UTENSILIOS DOMESTICO,UTILIDADES DOMESTICA,VIDEO
CLIENTE,,,,,,,,,,
1,0,0,1,0,0,0,0,0,0,0
65,0,0,0,0,0,0,0,0,1,0
68,1,0,0,0,0,0,0,0,0,0
75,1,0,0,0,0,0,0,0,0,0
76,0,0,0,0,0,0,0,0,1,0


In [ ]:
type(basket_lojas.head)

method

A etapa principal de contrução dos modelos e análise dos resultados.
Estabelecemos o suporte mínimo para cada modelo de acordo com a quantidade de operações de venda por canal, i.e., a frequência estabelecida para os canais Lojas e Site foi o percentual do total de operação dividido pelo número 100 de ocorrências e, para o canal de Televendas, por ter um volume muito menor de operação o suporte mínimo estabelecido foi de 10 ocorrências. Por isso realizamos a contagem dos objetos do atributo LINHA.

Logo, foi rodada no modelo Apriori a saída referente para cada canal após o *hot enconde*: basket_lojas; basket_site; basket_televendas; com os seus os seus respectivos Suportes mínimos estabelecidos. Configuramos também o parâmetro *use_colnames* como *True*, pois sem essa mudança o algoritmo retornaria o índice das colunas dos itens e não os seus respectivos nomes.

In [ ]:
frq_items = apriori(basket_lojas, min_support = 0.0002, use_colnames = True) 

A saída que se dá é a de um conjunto de itens com as suas frequências (Suporte).

In [ ]:
frq_items

,support,itemsets
0,0.230621,(ACESSORIOS)
1,0.190699,(APPLE)
2,0.162301,(AUDIO)
3,0.095470,(CELULAR)
4,0.028371,(GAMES)
...,...,...
136,0.000217,"(AUDIO, LINHA BRANCA, INFORMATICA, UTILIDADES ..."
137,0.000210,"(AUDIO, LINHA BRANCA, INFORMATICA, VIDEO)"
138,0.000206,"(AUDIO, INFORMATICA, VIDEO, UTILIDADES DOMESTICA)"
139,0.000849,"(AUDIO, LINHA BRANCA, VIDEO, UTILIDADES DOMEST..."


In [ ]:
type(frq_items)

pandas.core.frame.DataFrame

Fizemos uma varredura *(apply)* do dataframe e testamos a limitação dos itens para até duas saídas. Quando rodamos o algortimo *associations_rules* aumentamos para todos os casos o limite de 3.

In [ ]:
frq_items['lenght'] = frq_items['itemsets'].apply(lambda x: len(x))
frq_items = frq_items[(frq_items.lenght <= 3)]
frq_items

,support,itemsets,lenght
0,0.230621,(ACESSORIOS),1
1,0.190699,(APPLE),1
2,0.162301,(AUDIO),1
3,0.095470,(CELULAR),1
4,0.028371,(GAMES),1
5,0.075694,(INFORMATICA),1
6,0.127453,(LINHA BRANCA),1
7,0.002454,(UTENSILIOS DOMESTICO),1
8,0.161805,(UTILIDADES DOMESTICA),1
9,0.109704,(VIDEO),1


Agora vamos gerar o *dataframe* com as regras de associação, incluindo as métricas e estabelecendo a regra de interesse (*metric* igual a *Lift*), bem como o seu limite mínimo(*min_threshold* igual a  1).
O *dataframe* ainda foi ordenado  através do Suporte, seguido pelo *Lift*.


In [ ]:
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 

Avaliando a saída para Loja de imediato é percpetível que há uma relação entre as linhas Utensílios Doméstico e Utilidades Doméstica com uma Confiança de aproximadamente 0,3 – o que significa dizer que, em 30% dos casos, o item na posição RHS é adquirido quando ocorre a compra do item na posição LHS -, e um Lift aproximado de 1,85 – que significa que a probabilidade de se comprar os dois itens juntos é 1,85 vezes maior do que se comprar apenas o item na posição RHS. O suporte dessa operação é no valor de 0,007, e isso significa que a frequência de ocorrência da operação em si não é alta quando comparada às demais relações que seguem no quadro, exceto pela transação Utensílios Doméstico (LHS) e Linha Branca (RHS) o qual apresentam um suporte ainda menor.

In [ ]:
rules.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
6,(UTENSILIOS DOMESTICO),(UTILIDADES DOMESTICA),0.002454,0.161805,0.000734,0.299206,1.849178,0.000337,1.196065
1,(APPLE),(ACESSORIOS),0.190699,0.230621,0.051133,0.268132,1.162653,0.007153,1.051254
3,(UTENSILIOS DOMESTICO),(LINHA BRANCA),0.002454,0.127453,0.000600,0.244484,1.918230,0.000287,1.154902
0,(ACESSORIOS),(APPLE),0.230621,0.190699,0.051133,0.221717,1.162653,0.007153,1.039854
5,(VIDEO),(LINHA BRANCA),0.109704,0.127453,0.016324,0.148799,1.167482,0.002342,1.025078
9,(UTENSILIOS DOMESTICO),(VIDEO),0.002454,0.109704,0.000325,0.132392,1.206810,0.000056,1.026150
4,(LINHA BRANCA),(VIDEO),0.127453,0.109704,0.016324,0.128077,1.167482,0.002342,1.021072
2,(LINHA BRANCA),(UTENSILIOS DOMESTICO),0.127453,0.002454,0.000600,0.004706,1.918230,0.000287,1.002264
7,(UTILIDADES DOMESTICA),(UTENSILIOS DOMESTICO),0.161805,0.002454,0.000734,0.004537,1.849178,0.000337,1.002093
8,(VIDEO),(UTENSILIOS DOMESTICO),0.109704,0.002454,0.000325,0.002961,1.206810,0.000056,1.000509


Replicamos os mesmos métodos para os demais canais.
O Canal Site trouxe o entendimento de que os itens Acessórios e Utilidades Domésticas (LHS) e Apple (RHS) quando transacionados juntos retornaram com uma Confiança de pouco mais de 39% e um Lift de 1,22. Observando o Suporte, nesse caso, os demais itens transacionados seguem na ordem com a mesma grandeza. É possível ainda fazer uma observação em relação aos itens Vídeo e Utilidades Domésticas (LHS) e Linha Branca (RHS) que, embora tenham uma Confiança menor, possuem um Lift de 1,67.

In [ ]:
# Modelo do Site - 100
frq_items = apriori(basket_site, min_support = 0.0003, use_colnames = True) 
frq_items['lenght'] = frq_items['itemsets'].apply(lambda x: len(x))
frq_items = frq_items[(frq_items.lenght <= 3)]
# Coletando as Regras Inferidas em um Dataframe
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])

In [ ]:
rules.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
6,"(ACESSORIOS, UTILIDADES DOMESTICA)",(APPLE),0.000940,0.322516,0.000371,0.394265,1.222467,0.000067,1.118450
10,"(ACESSORIOS, VIDEO)",(APPLE),0.000947,0.322516,0.000354,0.373665,1.158595,0.000048,1.081665
0,"(ACESSORIOS, AUDIO)",(APPLE),0.001861,0.322516,0.000613,0.329710,1.022306,0.000013,1.010733
16,"(VIDEO, UTILIDADES DOMESTICA)",(LINHA BRANCA),0.002757,0.100762,0.000465,0.168704,1.674282,0.000187,1.081730
15,"(LINHA BRANCA, UTILIDADES DOMESTICA)",(VIDEO),0.002899,0.115990,0.000465,0.160465,1.383438,0.000129,1.052976
7,"(APPLE, UTILIDADES DOMESTICA)",(ACESSORIOS),0.003465,0.065628,0.000371,0.107004,1.630470,0.000143,1.046334
14,"(LINHA BRANCA, VIDEO)",(UTILIDADES DOMESTICA),0.004796,0.076986,0.000465,0.096978,1.259681,0.000096,1.022139
1,"(APPLE, AUDIO)",(ACESSORIOS),0.006721,0.065628,0.000613,0.091274,1.390783,0.000172,1.028222
4,"(APPLE, CELULAR)",(ACESSORIOS),0.003421,0.065628,0.000303,0.088670,1.351107,0.000079,1.025284
11,"(APPLE, VIDEO)",(ACESSORIOS),0.005147,0.065628,0.000354,0.068762,1.047764,0.000016,1.003366


Para o canal televendas, replicamos inicinalmente o método levando em considerção um Suporte mínimo de 100. Após o resultados, diminúimos esse valor de Suporte mínimo para 10 e rodamos o algortimo de de regras de associação.

In [ ]:
# Modelo do Televendas - 100
frq_items = apriori(basket_televendas, min_support = 0.0005, use_colnames = True) 
frq_items['lenght'] = frq_items['itemsets'].apply(lambda x: len(x))
frq_items = frq_items[(frq_items.lenght <= 3)]
# Coletando as Regras Inferidas em um Dataframe
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])

In [ ]:
rules.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ACESSORIOS),(APPLE),0.067327,0.359158,0.030941,0.459559,1.279544,0.00676,1.185775
1,(APPLE),(ACESSORIOS),0.359158,0.067327,0.030941,0.086147,1.279544,0.00676,1.020595


A  transação Acessórios (LHS) e Apple (RHS) possui o maior Suporte entre as 5 transações, além de uma Confiança de aproximadamente 46% e um Lift que se aproxima de 1,28. Ressalta-se que as transações desse canal apresentaram, mais uma vez, através da seleção de itens estabelecida no momento em que se trouxe os resultados, os maiores valores para a Confiança. Esse canal trouxe ainda, os dois maiores valores de Lift, na devida ordem, de pouco mais de 5,75 para a transação Informática e Utilidade Doméstica (LHS) e Linha Branca (RHS) e de aproximadamente 3,43 para a transação Áudio e Games (LHS) e Vídeo (RHS)

In [ ]:
# Modelo do Televendas - 10
frq_items = apriori(basket_televendas, min_support = 0.0000007, use_colnames = True) 
frq_items['lenght'] = frq_items['itemsets'].apply(lambda x: len(x))
frq_items = frq_items[(frq_items.lenght <= 3)]
# Coletando as Regras Inferidas em um Dataframe
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])

In [ ]:
rules.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
36,"(INFORMATICA, UTILIDADES DOMESTICA)",(LINHA BRANCA),0.000330,0.130363,0.000248,0.750000,5.753165,0.000205,3.478548
2,"(ACESSORIOS, LINHA BRANCA)",(APPLE),0.000248,0.359158,0.000165,0.666667,1.856191,0.000076,1.922525
24,"(AUDIO, GAMES)",(VIDEO),0.000165,0.145875,0.000083,0.500000,3.427602,0.000058,1.708251
6,"(ACESSORIOS, VIDEO)",(APPLE),0.000330,0.359158,0.000165,0.500000,1.392143,0.000046,1.281683
0,(ACESSORIOS),(APPLE),0.067327,0.359158,0.030941,0.459559,1.279544,0.006760,1.185775
34,"(LINHA BRANCA, INFORMATICA)",(UTILIDADES DOMESTICA),0.000660,0.052640,0.000248,0.375000,7.123824,0.000213,1.515776
14,"(ACESSORIOS, LINHA BRANCA)",(UTILIDADES DOMESTICA),0.000248,0.052640,0.000083,0.333333,6.332288,0.000069,1.421040
15,"(ACESSORIOS, UTILIDADES DOMESTICA)",(LINHA BRANCA),0.000248,0.130363,0.000083,0.333333,2.556962,0.000050,1.304455
10,"(ACESSORIOS, AUDIO)",(VIDEO),0.000248,0.145875,0.000083,0.333333,2.285068,0.000046,1.281188
30,"(AUDIO, LINHA BRANCA)",(VIDEO),0.000578,0.145875,0.000165,0.285714,1.958630,0.000081,1.195776
